In [11]:
import json
import os
import shutil
import subprocess
import zipfile
import json
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
def process_jsonl_files(cwes):
    base_dir = os.path.join(os.getcwd(), "raw_data_static_analyzer")

    for cwe in cwes:
        file_name = f"raw_data/{cwe}.jsonl"

        file_path = os.path.join(os.getcwd(), file_name)

        if not os.path.exists(file_path):
            print(f"File {file_name} does not exist. Skipping.")
            continue

        with open(file_path, 'r', encoding='utf-8') as file:
            num = 0
            for line in file:
                data = json.loads(line)
                num += 1
                func_name = data['func_name']
                func_name = f"output_sample_{num}_"
                file_name_before = func_name + data['file_name'].replace("/", "_")

                file_name_after = func_name + data['file_name'].replace("/", "_")


                dir_before = os.path.join(base_dir, cwe, "func_src_before")
                dir_after = os.path.join(base_dir, cwe, "func_src_after")


                os.makedirs(dir_before, exist_ok=True)
                os.makedirs(dir_after, exist_ok=True)


                path_before = os.path.join(dir_before, f"{file_name_before}")
                path_after = os.path.join(dir_after, f"{file_name_after}")


                with open(path_before, 'w', encoding='utf-8') as before_file:
                    before_file.write(data['func_src_before'])


                with open(path_after, 'w', encoding='utf-8') as after_file:
                    after_file.write(data['func_src_after'])

                print(f"Processed {func_name} into {path_before} and {path_after}")


In [8]:
cwes = ["cwe-078", "cwe-190", "cwe-416", "cwe-476"]
process_jsonl_files(cwes)

Processed output_sample_1_ into /Users/ic/Desktop/LLM-as-Static-Proxy-Test/raw_data_static_analyzer/cwe-078/func_src_before/output_sample_1_ajar.py and /Users/ic/Desktop/LLM-as-Static-Proxy-Test/raw_data_static_analyzer/cwe-078/func_src_after/output_sample_1_ajar.py
Processed output_sample_2_ into /Users/ic/Desktop/LLM-as-Static-Proxy-Test/raw_data_static_analyzer/cwe-078/func_src_before/output_sample_2_cinder_volume_drivers_storwize_svc.py and /Users/ic/Desktop/LLM-as-Static-Proxy-Test/raw_data_static_analyzer/cwe-078/func_src_after/output_sample_2_cinder_volume_drivers_storwize_svc.py
Processed output_sample_3_ into /Users/ic/Desktop/LLM-as-Static-Proxy-Test/raw_data_static_analyzer/cwe-078/func_src_before/output_sample_3_cinder_volume_drivers_storwize_svc.py and /Users/ic/Desktop/LLM-as-Static-Proxy-Test/raw_data_static_analyzer/cwe-078/func_src_after/output_sample_3_cinder_volume_drivers_storwize_svc.py
Processed output_sample_4_ into /Users/ic/Desktop/LLM-as-Static-Proxy-Test/raw_

In [ ]:
def zip_dir(path, ziph):
    """Zip the contents of an entire directory"""
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))

def zip_and_scan(base_dir):
    results_dir = os.path.join(base_dir, "static_analyzer_scans")
    logs_dir = os.path.join(base_dir, "logs", "scans")

    os.makedirs(results_dir, exist_ok=True)
 
    os.makedirs(logs_dir, exist_ok=True)
 
    for root, dirs, files in os.walk(base_dir):
        if 'func_src_before' in dirs or 'func_src_after' in dirs:
            for sub_dir in ['func_src_before', 'func_src_after']:
                full_dir_path = os.path.join(root, sub_dir)
                zip_file_path = f"{full_dir_path}.zip"

                with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
                    zip_dir(full_dir_path, zipf)

                scan_name = f"{os.path.basename(root)}_{sub_dir}_new_Scan"
                command = f"bash run_codeguru_security.sh {scan_name} {zip_file_path} 'us-east-1'"
                
 
                log_file_path = os.path.join(logs_dir, f"{scan_name}.log")
                with open(log_file_path, 'w') as log_file:
                    print(f"Running command: {command}")
                    result = subprocess.run(command, shell=True, text=True, stdout=log_file, stderr=subprocess.STDOUT)


 
                if result.returncode != 0:
                    print(f"Error running command, see log {log_file_path} for details.")

 
                json_result_file = f"{scan_name}.json"
                if os.path.exists(json_result_file):
                    shutil.move(json_result_file, os.path.join(results_dir, json_result_file))

    print("All directories processed and results moved.")



In [ ]:
base_dir = "raw_data_static_analyzer"
 
zip_and_scan(base_dir)

Running command: bash run_codeguru_security.sh cwe-190_func_src_before_new_Scan raw_data_static_analyzer/cwe-190/func_src_before.zip 'us-east-1'
Running command: bash run_codeguru_security.sh cwe-190_func_src_after_new_Scan raw_data_static_analyzer/cwe-190/func_src_after.zip 'us-east-1'
Running command: bash run_codeguru_security.sh cwe-416_func_src_before_new_Scan raw_data_static_analyzer/cwe-416/func_src_before.zip 'us-east-1'
Running command: bash run_codeguru_security.sh cwe-416_func_src_after_new_Scan raw_data_static_analyzer/cwe-416/func_src_after.zip 'us-east-1'
Running command: bash run_codeguru_security.sh cwe-078_func_src_before_new_Scan raw_data_static_analyzer/cwe-078/func_src_before.zip 'us-east-1'
Running command: bash run_codeguru_security.sh cwe-078_func_src_after_new_Scan raw_data_static_analyzer/cwe-078/func_src_after.zip 'us-east-1'
Running command: bash run_codeguru_security.sh cwe-476_func_src_before_new_Scan raw_data_static_analyzer/cwe-476/func_src_before.zip 'us

In [ ]:
def read_files_from_directory(directory_path):

    try:
        return {f for f in os.listdir(directory_path) if not f.startswith('.')}
    except FileNotFoundError:
        print(f"Directory not found: {directory_path}")
        return set()

def read_json_results(json_file):

    results = {}
 
    try:
        with open(json_file, 'r') as file:
            data = json.load(file)

        findings = data.get('findings', [])
        for finding in findings:
 
            path = finding['vulnerability']['filePath']['path']
            vulnerability = finding['vulnerability']['relatedVulnerabilities']
 
            if path in results:
                results[path].extend(vulnerability)
            else:
                results[path] = [vulnerability][0]
        return results
        
    except FileNotFoundError:
        print(f"File not found: {json_file}")
        return set()
    except Exception as e:
        print(f"Error reading {json_file}: {e}")
        return set()

def compute_metrics(before_files, after_files, predicted_paths, cwe):

    if cwe == "cwe-078":
        cwe = "cwe-78"
    
    tp = 0
    fp = 0
    
    for path in predicted_paths:
        if 'func_src_before' in path:
            if cwe in [element.lower() for element in predicted_paths[path]]:
                tp += 1
                
    for path in predicted_paths:
        if 'func_src_after' in path:
            if cwe in [element.lower() for element in predicted_paths[path]]:
                fp += 1
                
    fn = len(before_files) - tp
    tn = len(after_files) - fp

    total = tp + fp + fn + tn
    accuracy = (tp + tn) / total if total > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return accuracy, precision, recall, f1, tp, fp, tn, fn

def compute_pairwise_accuracy(before_predicted_paths, after_predicted_paths, before_files, after_files, cwe):

    if not before_files:
        return 0

    correct_pairs = 0
    for filename in before_predicted_paths:

 
        if filename not in after_predicted_paths and filename in after_files:

 
            correct_pairs += 1
            

 
    print("Correct Pairs...", correct_pairs)
    return correct_pairs / len(before_files)

 
base_path = "/Users/ic/Desktop/LLM-as-Static-Proxy-Test"
cwe_dir = os.path.join(base_path, "raw_data_static_analyzer")
scan_dir = os.path.join(cwe_dir, "static_analyzer_scans")

results = {}

 
#cwe_directories = [d for d in os.listdir(cwe_dir) if os.path.isdir(os.path.join(cwe_dir, d))]
cwe_directories = [d for d in os.listdir(cwe_dir) if os.path.isdir(os.path.join(cwe_dir, d)) and d.startswith("cwe-")]


for cwe in cwe_directories:
    print(f"Processing CWE {cwe}")
    
    path_before = os.path.join(cwe_dir, cwe, 'func_src_before')
    path_after = os.path.join(cwe_dir, cwe, 'func_src_after')

    before_files = read_files_from_directory(path_before)
    after_files = read_files_from_directory(path_after)
 
    print(f"  Number of files in func_src_before: {len(before_files)}")
    print(f"  Number of files in func_src_after: {len(after_files)}")

 
    json_files = [os.path.join(scan_dir, f) for f in os.listdir(scan_dir)
                  if f.startswith(cwe) and f.endswith('.json')]
 
    predicted_paths = {}
    for json_file in json_files:
        print(read_json_results(json_file))
        predicted_paths.update(read_json_results(json_file))

    print("all.....", predicted_paths)
    before_predicted_paths = []
    after_predicted_paths = []
    for path in predicted_paths:
        if "func_src_before" in path:
            before_predicted_paths.append(path.split("/")[1])
        if "func_src_after" in path:
            after_predicted_paths.append(path.split("/")[1])

    print("-------------------------")
    print("before predicted paths...", before_predicted_paths)
    print("after predicted paths...", after_predicted_paths)
    print("-------------------------")
    print("Predicted paths.....", predicted_paths)
 
    accuracy, precision, recall, f1, tp, fp, tn, fn = compute_metrics(before_files, after_files, predicted_paths, cwe)
    pairwise_accuracy = compute_pairwise_accuracy(before_predicted_paths,after_predicted_paths, before_files, after_files, cwe)
 
    results[cwe] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'True Positives': tp,
        'False Positives': fp,
        'True Negatives': tn,
        'False Negatives': fn,
        'Pairwise Accuracy': pairwise_accuracy,
        'Total Files in func_src_before': len(before_files),
        'Total Files in func_src_after': len(after_files)
    }

print("\nResults Summary:")
for cwe, metrics in results.items():
    print(f"CWE {cwe}:")
    for key, value in metrics.items():
        print(f"  {key}: {value}")
    print()
 

Processing CWE cwe-190
  Number of files in func_src_before: 38
  Number of files in func_src_after: 38
{'func_src_after/output_sample_15_src_delta.c': ['CWE-125'], 'func_src_after/output_sample_17_deps_lua_src_lua_struct.c': ['CWE-125']}
{'func_src_before/output_sample_15_src_delta.c': ['CWE-125'], 'func_src_before/output_sample_17_deps_lua_src_lua_struct.c': ['CWE-125']}
all..... {'func_src_after/output_sample_15_src_delta.c': ['CWE-125'], 'func_src_after/output_sample_17_deps_lua_src_lua_struct.c': ['CWE-125'], 'func_src_before/output_sample_15_src_delta.c': ['CWE-125'], 'func_src_before/output_sample_17_deps_lua_src_lua_struct.c': ['CWE-125']}
-------------------------
before predicted paths... ['output_sample_15_src_delta.c', 'output_sample_17_deps_lua_src_lua_struct.c']
after predicted paths... ['output_sample_15_src_delta.c', 'output_sample_17_deps_lua_src_lua_struct.c']
-------------------------
Predicted paths..... {'func_src_after/output_sample_15_src_delta.c': ['CWE-125'], '